In [ ]:
#DALL-E 사용하기 // 예시

# client = OpenAI() #openAI에 요청을 보내고 응답을 받는데 사용되는 코드

# dalle_response = client.Image.create(  #dall-e 이미지 생성 api호출
#     model="dall-e-3", #모델 선택
#     prompt="siamese cat", #DALL-E 모델에 전달할 프롬포트 (GPT-3.5모델 응답 사용)
#     size="1024x1024", #이미지크기
#     quality="standard", #이미지퀄리디 설정 #hd는 더 높은 퀄리티 / but 많은 비용과 시간
#     n=1,
# )

NameError: name 'OpenAI' is not defined

In [ ]:
!pip install openai==1.10.0 gradio==4.0.2  #DALL-E봇 라이브러리 설치
!pip install langchain==0.1.4 langchain-openai==0.0.5 #번역기능위한 langchain 설치

In [ ]:
import gradio as gr
import openai
import os

# 번역기능 위함
from langchain_openai import ChatOpenAI
from langchain_core.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

#DALL KEY
openai.api_key = "" #DALL E
os.environ["OPENAI_API_KEY"]= openai.api_key

client=openai.OpenAI()


# 번역기능 함수
def translate_text(text):
    chat = ChatOpenAI(temperature=0)

    template="You are a helpful assistant that translates {input_language} to {output_language}."
    system_message_prompt = SystemMessagePromptTemplate.from_template(template)

    human_template="{text}"
    human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

    chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

    response = chat(
    chat_prompt.format_prompt(
        input_language="korean", output_language="english", text=text
    ).to_messages()
)

    return response.content


# GPT+ DALL-E를 사용하여 응답 생성
def generate_response(prompt, size):
  translated_prompt = translate_text(prompt)

  gpt_prompt = [
      {"role":"system", "content": "Imagine the details of the input's appearence. Please respond breifly"},
      {"role":"user","content":translated_prompt}
  ]

  gpt_response=client.chat.completions.create(
      model="gpt-3.5-turbo",
      messages=gpt_prompt
  )

  gpt_prompt_result = gpt_response.choices[0].message.content

  dalle_response = client.images.generate(
      model="dall-e-3",
      prompt=gpt_prompt_result,
      size=size,
      quality="standard",
      n=1,
  )

  image_url = dalle_response.data[0].url
  return prompt, image_url


image_sizes = gr.Dropdown(choices=["1024x1024", "1024x1792", "1792x1024"], label="이미지 크기")

iface = gr.Interface(
    fn=generate_response,
    inputs=["text",image_sizes],
    outputs = ["text","image"],
    title = "ES's DALL-E봇 ",
    description = "사용자 프롬포트를 기반으로 ChatGPT와  DALL-E를 사용하여 이미지를 제작합니다."
)

iface.launch(debug=True, share=True)


IMPORTANT: You are using gradio version 4.0.2, however version 4.29.0 is available, please upgrade.
--------
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://6bdd3abf2fdb9fb7ad.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/gradio/queueing.py", line 427, in call_prediction
    output = await route_utils.call_process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/route_utils.py", line 232, in call_process_api
    output = await app.get_blocks().process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1497, in process_api
    result = await self.call_function(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1119, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "/usr/local/lib/python3.10/dist-packages/anyio/to_thread.py", line 33, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://6bdd3abf2fdb9fb7ad.gradio.live
